<h1>Next Word Predictor</h1>
<p>Data collection and cleaning done in data preprocessing notebook</p>

<h3>Load packages

In [46]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

<h2>Load Data

In [48]:
data=pd.read_csv("gullivers_travels.csv") 

In [49]:
data.head()

,paragraphs
0,i desired the secretary to present my humble d...
1,but not to detract from a nation to which dur...
2,he consented and i immediately stripped mysel...
3,i then desired the governor to call up descart...
4,when i formerly hinted to you something of th...


In [50]:
rows = list(data['paragraphs'])
len(rows)

19997

In [51]:
rows[1]

' but not to detract from a nation to which during my life i shall acknowledge myself extremely obliged it must be allowed that whatever this famous tower wants in height is amply made up in beauty and strength for the walls are near a hundred feet thick built of hewn stone whereof each is about forty feet square and adorned on all sides with statues of gods and emperors cut in marble larger than the life placed in their several niches'

<h2> Collect unique words in dataset

In [53]:
words = []

for i in rows:
    row = i.replace('\n','').split()
    for word in row:
        words.append(word)

        
words = set(words)
words = list(words)
words[:20]

['men',
 'remarkable',
 'collected',
 'objects',
 'seats',
 'wit',
 'informations',
 'integrity',
 'rogues',
 'temperance',
 'uneasiness',
 'sufficient',
 'travelled',
 'chinese',
 'canal',
 'upright',
 'reflect',
 'fairer',
 'shortening',
 'earnestness']

In [54]:
words_df = pd.DataFrame(data=words, index=None, columns=['words'])
words_df.to_csv('words.csv', index=False)

In [55]:
len(words)

8147

<h2> Split data into train and test sets

In [56]:
#target and feature sets

x = []
y = []

for j in rows:
    sentence = j.split()
    last_word = sentence[-1] #pluck last word
    sentence.pop() #remove last word
    sentence = " ".join(sentence) 
    x.append(sentence) #add sentence, minus last word to x

    encoded_word = [0]*(len(words)) 
    encoded_word[words.index(last_word)]=1 #encode last word
    y.append(encoded_word)
  

In [57]:
len(x), len(y)

(19997, 19997)

In [58]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.25, random_state=55)


In [59]:
xtrain[0]

'but hurried on by the precipitancy of youth and having his imperial majestys license to pay my attendance upon the emperor of blefuscu i took this opportunity before the three days were elapsed to send a letter to my friend the secretary signifying my resolution of setting out that morning for blefuscu pursuant to the leave i had got and without waiting for an answer i went to that side of the island where our fleet'

<h2> Universal Sentence Encoder

In [60]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [61]:
xtrain = embed(xtrain) #embed
xtest = embed(xtest) #embed
#covert to numpy arrays/matrices
xtrain = xtrain.numpy()
xtest = xtest.numpy()
ytest = np.array(ytest) 
ytrain = np.array(ytrain)

In [62]:
xtrain.shape, ytrain.shape

((14997, 512), (14997, 8147))

In [63]:
xtrain[10]

array([-0.04899583, -0.03114763,  0.00227157,  0.00442289, -0.05547805,
        0.06363542,  0.05368468,  0.01622464,  0.06673943,  0.04664867,
       -0.04496281, -0.05015457,  0.02276505,  0.0443054 , -0.08912884,
       -0.05177389, -0.0866108 , -0.06640136,  0.01699785, -0.06158642,
        0.06076851,  0.00676135, -0.07722596, -0.02966137,  0.02902962,
       -0.08251665, -0.06471777, -0.03177203, -0.0776021 ,  0.03117883,
        0.00563257,  0.0669057 , -0.05106832, -0.04172863, -0.0039697 ,
        0.04440599,  0.01250168, -0.0678933 , -0.03196873, -0.04358714,
       -0.00074472,  0.08387917,  0.03252276,  0.02008015,  0.00449204,
        0.03164991, -0.03134472,  0.01680997, -0.05335777, -0.07710892,
       -0.03624308,  0.01026028,  0.02407982,  0.02771026, -0.02083948,
       -0.04527139,  0.00093273,  0.05376001, -0.0222789 ,  0.00753828,
        0.04776055,  0.04062064, -0.02747052, -0.01985496, -0.06104327,
       -0.0201566 , -0.01087866, -0.05189257,  0.04879563, -0.06

<h2>Model Building

In [64]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Activation
from keras.models import Sequential

In [65]:
model = Sequential()

model.add(Dense(512, input_shape=[512], activation = 'relu'))
model.add(Dense(units=len(words), activation = 'sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 8147)              4179411   
Total params: 4,442,067
Trainable params: 4,442,067
Non-trainable params: 0
_________________________________________________________________


In [66]:
model.fit(xtrain, ytrain, batch_size=350, shuffle=True, epochs=30, validation_data=(xtest, ytest))

Epoch 1/30
43/43 [==============================] - 6s 133ms/step - loss: 8.6460 - acc: 0.0228 - val_loss: 6.9886 - val_acc: 0.0232
Epoch 2/30
43/43 [==============================] - 5s 111ms/step - loss: 6.7427 - acc: 0.0250 - val_loss: 6.4615 - val_acc: 0.0238
Epoch 3/30
43/43 [==============================] - 5s 116ms/step - loss: 6.1467 - acc: 0.0362 - val_loss: 5.7361 - val_acc: 0.0468
Epoch 4/30
43/43 [==============================] - 5s 109ms/step - loss: 5.2791 - acc: 0.0926 - val_loss: 4.7381 - val_acc: 0.1862
Epoch 5/30
43/43 [==============================] - 5s 109ms/step - loss: 4.1212 - acc: 0.3121 - val_loss: 3.5549 - val_acc: 0.4060
Epoch 6/30
43/43 [==============================] - 5s 118ms/step - loss: 2.8909 - acc: 0.5558 - val_loss: 2.5560 - val_acc: 0.5954
Epoch 7/30
43/43 [==============================] - 5s 116ms/step - loss: 1.9676 - acc: 0.7060 - val_loss: 1.8819 - val_acc: 0.6912
Epoch 8/30
43/43 [==============================] - 5s 122ms/step - loss: 1.

<h2>Trained model predictions</h2>
<p>Model was trained on Gulliver's Travels, and therefore is biased to the language used in the novel

In [67]:
def next_word(given_words):
    prediction = model.predict(x=embed([given_words]).numpy())
    index = np.argmax(prediction[-1])
    pred = words[index]
    given_words += ' ' + words[index]
      
    print(given_words, '\nnext word:', pred)

In [77]:
#example
single_text = "he was on the"

next_word(single_text)

he was on the island 
next word: island


<h2>Saving model

In [78]:
model.save("next_word_model.h5")

In [79]:
model = keras.models.load_model('next_word_model.h5')